In [ ]:
import os
import torch
import numpy as np
import pandas as pd

In [ ]:
class RecurrencePlot:
    def __init__(self, input_address, output_address, iteration=0, window_size=160, start=0, end_point=24960,
                 color="rainbow",
                 res=1024):
        self.input_address = input_address
        self.output_address = output_address
        self.window_size = window_size
        self.start = start
        self.end_point = end_point
        self.chunks_dict = {}
        self.color = color
        self.res = res
        self.iteration = iteration
        self.load_data()
        self.Saveon()

    def recurrence_plot_magnitude(self, A):
        N = len(A)
        distance_matrix = np.zeros((N, N))
        for i in range(N):
            for j in range(N):
                distance_matrix[i, j] = np.abs(A[i] - A[j])
        return distance_matrix

    def load_data(self):
        chunks_dict = {}
        address = self.input_address
        pre_df = pd.read_csv(address)
        df = pre_df.iloc[15:, [1, 3, 5]].values.astype(float)
        df = df[:, 0]
        for i in range(self.start, self.end_point, self.window_size):
            chunks_dict[i // self.window_size] = df[i:i + self.window_size]
        self.chunks_dict = chunks_dict
        os.makedirs(self.output_address, exist_ok=True)

    def Saveon(self):
        os.makedirs(self.output_address, exist_ok=True)  # Ensure output directory exists

        for key, chunk in self.chunks_dict.items():
            # Generate the recurrence plot (distance matrix)
            matrix = self.recurrence_plot_magnitude(chunk)

            # Convert the matrix to a PyTorch tensor
            tensor = torch.tensor(matrix, dtype=torch.float32)

            # Construct filename with iteration and input file name for clarity
            base_filename = os.path.basename(self.input_address).replace('.csv', '')
            filename_tensor = f'{base_filename}_iter{key + self.iteration}.pt'
            full_path_tensor = os.path.join(self.output_address, filename_tensor)

            # Save the tensor and handle errors
            try:
                torch.save(tensor, full_path_tensor)
                print(f"Saved PyTorch tensor: {full_path_tensor}, Tensor shape: {tensor.shape}")
            except Exception as e:
                print(f"Error saving tensor for chunk {key}: {e}")

In [ ]:
directory_out = r"F:\TheLis\1"
directory_in = r"F:\Thesis\Data Set\Machin_Train_URF\Machin_Train_URF\0\1420"
limit, count = 8, 0
iteration = 0

for filename in sorted(os.listdir(directory_in)):
    if filename.endswith(".CSV") or filename.endswith(".csv"):  # Ensure only CSV files are processed
        if count == limit:
            break
        input_file_path = os.path.join(directory_in, filename)  # Create full path for each file
        RecurrencePlot(input_file_path, directory_out, iteration=iteration)

        count += 1
        iteration = count * 156
        print(input_file_path)